In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install qiskit qiskit-machine-learning qiskit_algorithms qiskit-aer pandas scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 16.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.algorithms import VQC
from qiskit_algorithms.optimizers import COBYLA, SPSA,ADAM
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.primitives import Sampler
import matplotlib.pyplot as plt
from IPython.display import clear_output


In [ ]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against Iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()


In [ ]:
file_path = 'pd_speech_features.csv'

In [ ]:
data = pd.read_csv(file_path, skiprows=1)
print(data.shape)
print(data.head())


(756, 755)
   id  gender      PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0   0       1  0.85247  0.71826  0.57227        240               239   
1   0       1  0.76686  0.69481  0.53966        234               233   
2   0       1  0.85083  0.67604  0.58982        232               231   
3   1       0  0.41121  0.79672  0.59257        178               177   
4   1       0  0.32790  0.79782  0.53028        236               235   

   meanPeriodPulses  stdDevPeriodPulses  locPctJitter  ...  \
0          0.008064            0.000087       0.00218  ...   
1          0.008258            0.000073       0.00195  ...   
2          0.008340            0.000060       0.00176  ...   
3          0.010858            0.000183       0.00419  ...   
4          0.008162            0.002669       0.00535  ...   

   tqwt_kurtosisValue_dec_28  tqwt_kurtosisValue_dec_29  \
0                     1.5620                     2.6445   
1                     1.5589                     3.6107   


In [ ]:
X = data.drop(['id', 'class'], axis=1)
y = data['class'].values

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
#split into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}")

Train shape: (604, 753), Val shape: (152, 753)


In [ ]:
##function to Train vqc with different configurations
def train_vqc(num_qubits, feature_map_type, ansatz_type, reps, optimizer_type, add_noise=False):
    #dimensionality reduction to num_qubits
    pca = PCA(n_components=num_qubits)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)

    #feature map
    if feature_map_type == 'ZZ':
        feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement='linear')
    elif feature_map_type == 'Z':
        feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=2)

    #ansatz
    if ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(num_qubits=num_qubits, reps=reps)
    elif ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(num_qubits=num_qubits, reps=reps)
    elif optimizer_type == 'Adam':
         from qiskit.algorithms.optimizers import ADAM
         optimizer = ADAM(maxiter=100, lr=0.1)
    #optimizer
    if optimizer_type == 'COBYLA':
        optimizer = COBYLA(maxiter=100)
    elif optimizer_type == 'SPSA':
        optimizer = SPSA(maxiter=100)


    if add_noise:
        noise_model = NoiseModel()
        dep_error = depolarizing_error(0.01, 1)
        noise_model.add_all_qubit_quantum_error(dep_error, ['rx', 'ry', 'rz'])
        sampler = Sampler(options={"noise_model": noise_model})
    else:
        sampler = Sampler()

    # VQC
    global objective_func_vals
    objective_func_vals = []
    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        sampler=sampler,
        callback=callback_graph
    )

    vqc.fit(X_train_pca, y_train)
    y_pred = vqc.predict(X_val_pca)
    acc = accuracy_score(y_val, y_pred)

    qc = feature_map.compose(ansatz).decompose()
    num_params = ansatz.num_parameters
    depth = qc.depth()
    qubits = qc.num_qubits

    print(f"Config: Qubits={num_qubits}, FM={feature_map_type}, Ansatz={ansatz_type}, Reps={reps}, Opt={optimizer_type}, Noise={add_noise}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Parameters: {num_params}, Depth: {depth}, Qubits: {qubits}")

    return vqc, acc, num_params, depth, qubits, pca

In [ ]:
configs = [
     (4, 'ZZ', 'RealAmplitudes', 6, 'SPSA', False),
    (4, 'Z', 'RealAmplitudes', 6, 'SPSA', False),
    (4, 'ZZ', 'EfficientSU2', 6, 'COBYLA', False),
    (6, 'ZZ', 'RealAmplitudes', 8, 'SPSA', False),
    (4, 'Z', 'RealAmplitudes', 6, 'SPSA', False),
        (4, 'Z', 'RealAmplitudes', 10, 'SPSA', False),


]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
best_model = None
best_acc = 0
best_metrics = None
best_pca = None

for config in configs:
    model, acc, params, depth, qubits, pca = train_vqc(*config)
    y_pred = model.predict(pca.transform(X_val))
    prec = precision_score(y_val, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_val, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_val, y_pred, average='weighted', zero_division=0)

    print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")


    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_metrics = (params, depth, qubits)
        best_pca = pca

print(f"Best Model Accuracy: {best_acc:.4f}")
print(f"Best Metrics: Parameters={best_metrics[0]}, Depth={best_metrics[1]}, Qubits={best_metrics[2]}")


/tmp/ipython-input-741553969.py:7: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  model, acc, params, depth, qubits, pca = train_vqc(*config)


Config: Qubits=4, FM=Z, Ansatz=RealAmplitudes, Reps=6, Opt=SPSA, Noise=False
Accuracy: 0.6776
Parameters: 28, Depth: 24, Qubits: 4
Precision: 0.5630, Recall: 0.6776, F1: 0.6095


/tmp/ipython-input-741553969.py:7: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  model, acc, params, depth, qubits, pca = train_vqc(*config)


Config: Qubits=4, FM=Z, Ansatz=RealAmplitudes, Reps=10, Opt=SPSA, Noise=False
Accuracy: 0.7500
Parameters: 44, Depth: 36, Qubits: 4
Precision: 0.6859, Recall: 0.7434, F1: 0.6562
Best Model Accuracy: 0.7500
Best Metrics: Parameters=44, Depth=36, Qubits=4
